# Proyecto Individual 1 
## *Machine Learning Operations (MLOps)*

### 1. Ingesta de datos

**1.1 Importamos librerías**

Importamos las librerias que vamos a usar durante el desarrollo del proyecto.

In [2]:
import pandas as pd 
import numpy as np
import json
import gzip
import ast

**1.2 Leemos los datos**

Utilizaremos 3 archivos: steam_games, user_reviews y users_items. 

In [3]:
ruta1 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/steam_games.json'
ruta2 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/user_reviews.json.gz'
ruta3 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/users_items.json.gz'

Mediante el uso de Pandas, leemos el archivo 'steam_games' y lo asignamos a un nombre identificatorio.

1. **steam_games**

In [4]:
df_games = pd.read_json(ruta1, lines=True)

In [5]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0


Para leer los archivos user_reviews y users_items, utilizamos otro método.

2. **user_reviews**

In [6]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

for i in gzip.open(ruta2):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_reviews = pd.DataFrame(info) 

In [7]:
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


3. **users_items**

In [8]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

for i in gzip.open(ruta3):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_items = pd.DataFrame(info) 

In [9]:
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### 2. Tratamiento de datos

**2.1 Desanidar columnas**

Luego de leer los datos, se observa que los tres dataframes presentan columnas con datos anidados.
 
Para ello, vamos a utilizar las funciones .explode y .json_normalize, con el objetivo de desanidar la informacion y convertir cada clave a columna.

Luego de desanidar, concatenamos las columnas del data frame original con las columnas desanidadas. 

1.  Del dataframe **df_games** desanidamos la informacion que se encuentra en la columna 'items'.

In [10]:
df_explode = df_games.explode('items').reset_index()

games_items = pd.json_normalize(df_explode['items'])

df_games_full = pd.concat([df_games.drop('items', axis=1), games_items], axis=1)


In [13]:
df_games_full.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,...,id,metascore,developer,user_id,steam_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,76561197970982479,7.656120e+16,277.0,10,Counter-Strike,6.0,0.0
1,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,js41637,7.656120e+16,888.0,20,Team Fortress Classic,0.0,0.0
2,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,evcentric,7.656120e+16,137.0,30,Day of Defeat,7.0,0.0
3,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,Riot-Punch,7.656120e+16,328.0,40,Deathmatch Classic,0.0,0.0
4,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,doctr,7.656120e+16,541.0,50,Half-Life: Opposing Force,0.0,0.0


2. Del dataframe **df_reviews** desanidamos la informacion que se encuentra en la columna 'reviews'.

In [74]:
df_explode = df_reviews.explode('reviews').reset_index()

reviews = pd.json_normalize(df_explode['reviews'])

df_reviews_full = pd.concat([df_reviews.drop('reviews', axis=1), reviews], axis=1)

In [75]:
df_reviews_full.head(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


3. Del dataframe **df_items** desanidamos la informacion que se encuentra en la columna 'items'.

In [16]:
df_explode = df_items.explode('items').reset_index()

items = pd.json_normalize(df_explode['items'])

df_items_full = pd.concat([df_items.drop('items', axis=1), items], axis=1)

In [17]:
df_items_full.head(5)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,20,Team Fortress Classic,0.0,0.0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,30,Day of Defeat,7.0,0.0
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,40,Deathmatch Classic,0.0,0.0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,50,Half-Life: Opposing Force,0.0,0.0


### 3. Feature Engineering

En el planteamiento de la problematica del proyecto, se solicita la creación de la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP. La escala a tener en cuenta es la siguiente: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo.

Para ello analizaremos la columna 'reviews' del dataframe 'df_reviews_full' obtenido en el paso 2.1

Utilizaremos la librería **TextBlob** 

In [65]:
from textblob import TextBlob

In [66]:
def calcular_sentimiento_textblob(texto):
    # Utiliza TextBlob para calcular el sentimiento del texto
    analysis = TextBlob(texto)
    # El atributo sentiment.polarity devuelve la polaridad del sentimiento (entre -1 y 1)
    polaridad = analysis.sentiment.polarity
    return polaridad

def asignar_puntaje(polaridad):
    # Asigna 0 si la polaridad es negativa, 1 si es neutral y 2 si es positiva
    if polaridad < -0.1:
        return 0  # Negativo
    elif polaridad > 0.1:
        return 2  # Positivo
    else:
        return 1  # Neutral


In [76]:
df_reviews_full['polaridad'] = df_reviews_full['review'].astype(str).apply(calcular_sentimiento_textblob)
df_reviews_full['sentimiento'] = df_reviews_full['polaridad'].apply(asignar_puntaje)

In [78]:
df_reviews_full

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,polaridad,sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500,2
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0.050000,1
3,doctr,http://steamcommunity.com/id/doctr,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,0.114583,2
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,-0.036111,1
...,...,...,...,...,...,...,...,...,...,...,...
59328,NaN,NaN,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,0.233333,2
59329,NaN,NaN,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,0.142500,2
59330,NaN,NaN,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,0.087500,1
59331,NaN,NaN,,Posted July 20.,,730,No ratings yet,True,:D,1.000000,2


### 4. Disponibilización de datos (API)

### 5. Deployment

### 6. Analisis exploratorio

### 7. Modelamiento